In [2]:
import math
import json
import logging
# import numpy as np
from decimal import *
from web3 import Web3

WETH_ERC20 = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'
PEPE_ERC20 = '0xfb66321D7C674995dFcC2cb67A30bC978dc862AD'

PEPE_UNI_V2 = '0x076a3e1500f3110D8F4445D396A3d7cA6D0Ca269'
PEPE_UNI_V3 = '0xd738e6a2ef2846a643dc68092ad0fd7f5a8eb6f8'

UNI_V3_QUOTER = "0xb27308f9F90D607463bb33eA1BeBb41C27CE5AB6"
UNI_V2_ROUTER = "0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D"

logging.basicConfig(level=logging.INFO)

In [3]:
with open("../abi/uniswap_v2.json") as f:
    uni_v2_abi = json.load(f)

with open("../abi/uniswap_v3.json") as f:
    uni_v3_abi = json.load(f)

web3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))

pepe_uni_v2 = web3.eth.contract(
    address=PEPE_UNI_V2,
    abi=uni_v2_abi,
)

reserves = pepe_uni_v2.functions.getReserves().call()
pool_v2_eth = reserves[0]
pool_v2_pepe = reserves[1]

v2_price = pool_v2_pepe/pool_v2_eth
print(v2_price)

pepe_uni_v3 = web3.eth.contract(
    address=web3.to_checksum_address(PEPE_UNI_V3),
    abi=uni_v3_abi,
)
slot0 = pepe_uni_v3.functions.slot0().call()

cur_tick = slot0[1]

v3_price = math.pow(1.0001, cur_tick)
print(v3_price)

27874522752.222176
29111388743.317177


In [ ]:
print(cur_tick)
# print(cur_tick//200 * 200)
tick_lower = cur_tick//200 * 200
tick_upper = tick_lower + 200
tick_liquidity = pepe_uni_v3.functions.ticks(240800).call()
print(f"tick liquidity: {tick_liquidity}")

liquidity = pepe_uni_v3.functions.liquidity().call()
print(f"pool liquidity: {liquidity}")

In [ ]:
tick_lower_price = math.pow(1.0001, tick_lower)
tick_upper_price = math.pow(1.0001, tick_upper)
print(tick_lower_price/tick_upper_price)

In [ ]:
sqrt_price = slot0[0]
pool_v3_eth = liquidity / (sqrt_price>>96)
pool_v3_pepe = liquidity * (sqrt_price>>96)
print(f"v3 eth : {int(pool_v3_eth)}, {pool_v3_eth/10**18}")
print(f"v3 pepe: {pool_v3_pepe}, {pool_v3_pepe/10**18}")

print(f"v2 eth : {pool_v2_eth}, {pool_v2_eth/10**18}")
print(f"v2 pepe: {pool_v2_pepe}, {pool_v2_pepe/10**18}")

In [1]:
class ArbiTest:
    V3_FEE = 10000

    def __init__(self):
        web3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
        # self.pepe_erc20_addr = PEPE_ERC20
        # self.weth_erc20_addr = WETH_ERC20

        with open("../abi/uniswap_v3_quoter.json") as f:
            uni_v3_quoter_abi = json.load(f)
        
        with open("../abi/uniswap_v2_router.json") as f:
            uni_v2_router_abi = json.load(f)

        self.uni_v3_quoter = web3.eth.contract(
            UNI_V3_QUOTER,
            abi = uni_v3_quoter_abi,
        )

        self.uni_v2_router = web3.eth.contract(
            UNI_V2_ROUTER,
            abi = uni_v2_router_abi,
        )

    def get_arbitarge_profit(self, qty):
        pepe_out_from_v3 = self.get_pepe_out_from_v3(qty)
        # logging.info(f"pepe amount out from v3: {pepe_out_from_v3}")

        pepe_in_to_v2 = pepe_out_from_v3 - (pepe_out_from_v3//100)
        # logging.info(f"pepe amount in to v2: {pepe_in_to_v2}")

        eth_out_from_v2 = self.get_eth_out_from_v2(pepe_in_to_v2)
        # logging.info(f"eth amount out from v2: {eth_out_from_v2}")

        profit = eth_out_from_v2 - qty
        return profit

    # "function quoteExactInputSingle(address,address,uint24,uint256,uint160) view returns (uint256)"
    def get_pepe_out_from_v3(self, eth_amount_in):
        amount_out = self.uni_v3_quoter.functions.quoteExactInputSingle(
            WETH_ERC20,
            PEPE_ERC20,
            self.V3_FEE,
            eth_amount_in,
            0,
        ).call()
        return amount_out

    # "function getAmountsOut(uint256, address[]) view returns (uint256[])",
    def get_eth_out_from_v2(self, pepe_amount_in):
        result = self.uni_v2_router.functions.getAmountsOut(
            pepe_amount_in,
            [PEPE_ERC20, WETH_ERC20]
        ).call()
        eth_amount_out = result[1]
        return eth_amount_out

abr_test = ArbiTest()

NameError: name 'Web3' is not defined

In [38]:
# wei_qty = 105*10**17 # 0.2 eth
# wei_qty = 1057147387365399552
# profit = abr_test.get_arbitarge_profit(wei_qty)
# print(profit)

amount_in = int(0.6*10**18)
delta_qty = int(0.005*10**18)
max_qty = int(1.5*10**18)
while amount_in < max_qty:
# for i in range(20):
    amount_in += delta_qty
    # print(amount_in wei_qty)
    profit = abr_test.get_arbitarge_profit(amount_in)
    print(f"profit: {profit:18d}, amount_in: {Decimal(amount_in)/10**18:.3f}")
    # print(profit)

profit:   8964100549768943, amount_in: 0.605
profit:   9009336129044571, amount_in: 0.610
profit:   9054101502589613, amount_in: 0.615
profit:   9098396736134337, amount_in: 0.620
profit:   9142221895396762, amount_in: 0.625
profit:   9185577046082656, amount_in: 0.630
profit:   9228462253885546, amount_in: 0.635
profit:   9270877584486714, amount_in: 0.640
profit:   9312823103555204, amount_in: 0.645
profit:   9354298876747823, amount_in: 0.650
profit:   9395304969709144, amount_in: 0.655
profit:   9435841448071512, amount_in: 0.660
profit:   9475908377455040, amount_in: 0.665
profit:   9515505823467618, amount_in: 0.670
profit:   9554633851704914, amount_in: 0.675
profit:   9593292527750377, amount_in: 0.680
profit:   9631481917175238, amount_in: 0.685
profit:   9669202085538515, amount_in: 0.690
profit:   9706453098387016, amount_in: 0.695
profit:   9743235021255340, amount_in: 0.700
profit:   9779547919665881, amount_in: 0.705
profit:   9815391859128830, amount_in: 0.710
profit:   